In de database zijn de volgende indexen geplaatst
1.

In [ ]:
def optim_enkel_indexen(connection, column_names, jaar_1=2000, jaar_2=1990, lengte=75):
    query="""
    SELECT m.birthState, AVG(m.weight), AVG(m.height), AVG(bat.HR), AVG(pit.SV)
    FROM Master AS m,
        Pitching AS pit,
        Batting AS bat,
        HallOfFame AS hof
    WHERE pit.yearID = {}
        AND bat.yearID = {}}
        AND pit.playerID = m.playerID
        AND bat.playerID = m.playerID
        AND m.playerID = hof.playerID
        AND hof.yearID > {}
    GROUP BY m.birthState
    HAVING AVG(m.height) > {}
    ORDER BY m.birthState ASC;
    """.format(jaar_1, jaar_1, jaar_2, lengte)


In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
timings3=[]

# We voeren deze query 5 keer uit om een betere schatting te krijgen van de effectieve runtime.
for i in range(5):
    t1 = time.time() # Start time
    df_orig = query_to_optimize(c, column_names, jaar_1=2000, jaar_2=1990, lengte=75)
    t2 = time.time() # Stop time
    timings3.append(t2-t1) 
    
orig_runtime=np.mean(timings3) # De uiteindelijke runtime is een gemiddelde van 5 runs.

print('De originele query duurt: ', "{0:.2f}".format(orig_runtime),' seconden')

In [ ]:
# Maak hier je eerste optimalisatie
def optim_1(connection, column_names, jaar_1=2000, jaar_2=1990, lengte=75):
    # het opstellen 
    query="""
   
     SELECT bpm.BirthState, AVG(bpm.weight), AVG(bpm.height), AVG(bpm.HR), AVG(bpm.SV)
     FROM
        (SELECT m.playerID, m.BirthState, m.weight, m.height, bp.SV, bp.HR
         FROM   
            (SELECT p.playerID, p.SV, b.HR
             FROM   (SELECT bat.playerID, bat.yearID, bat.HR
                     FROM batting as bat
                    ) as b, 
                
                    (SELECT pit.yearID, pit.playerID, pit.SV
                     FROM pitching as pit
                    ) as p
                
             WHERE     p.yearID = {}
                   AND b.yearID = {}   
                   AND p.playerID = b.playerID
            ) as bp, 
        
        
            (SELECT mas.playerID, mas.BirthState, mas.weight, mas.height 
             FROM master as mas
            ) as m
        
        WHERE 
            bp.playerID = m.playerID
       ) as bpm,
       
       
        (SELECT hof.playerID
         FROM   halloffame as hof
         WHERE hof.yearID >{}
        ) as h
    WHERE h.playerID = bpm.playerID
       
    GROUP BY bpm.birthState
    HAVING AVG(bpm.height) > {}
    ORDER BY bpm.birthState ASC;
   
   
    """.format(jaar_1, jaar_1, jaar_2, lengte)

    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
timings2=[]

# We voeren deze query 5 keer uit om een betere schatting te krijgen van de effectieve runtime.
for i in range(5):
    t1 = time.time() # Start time
    df_orig = query_to_optimize(c, column_names, jaar_1=2000, jaar_2=1990, lengte=75)
    t2 = time.time() # Stop time
    timings2.append(t2-t1) 
    
orig_runtime=np.mean(timings2) # De uiteindelijke runtime is een gemiddelde van 5 runs.

print('De originele query duurt: ', "{0:.2f}".format(orig_runtime),' seconden')